In [1]:
library(repr)
library(tidyverse)
library(tidymodels)
library(readxl)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [20]:
# ignore
# knowledge_data <- read_excel("Data_User_Modeling_Dataset_Hamdi Tolga KAHRAMAN.xls", sheet = 2, range = "A1:F259")
# mean_data <- knowledge_data |>
#     group_by(UNS) |>
#     summarize(mean_stg = mean(STG),
#              mean_scg = mean(SCG),
#              mean_str = mean(STR),
#              mean_lpr = mean(LPR),
#              mean_peg = mean(PEG))
# mean_data

UNS,mean_stg,mean_scg,mean_str,mean_lpr,mean_peg
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
High,0.4216508,0.4231905,0.5016667,0.5012698,0.77253968
Low,0.3211446,0.3370000,0.4307229,0.4973494,0.23762651
Middle,0.3999773,0.3679205,0.5068182,0.3428409,0.54238636
very_low,0.3057917,0.1981250,0.3662500,0.3587500,0.09083333


In [6]:
?read_excel

read_excel {readxl},R Documentation
path,Path to the xls/xlsx file.
sheet,"Sheet to read. Either a string (the name of a sheet), or an integer (the position of the sheet). Ignored if the sheet is specified via range. If neither argument specifies the sheet, defaults to the first sheet."
range,"A cell range to read from, as described in cell-specification. Includes typical Excel ranges like ""B3:D87"", possibly including the sheet name like ""Budget!B2:G14"", and more. Interpreted strictly, even if the range forces the inclusion of leading or trailing empty rows or columns. Takes precedence over skip, n_max and sheet."
col_names,"TRUE to use the first row as column names, FALSE to get default names, or a character vector giving a name for each column. If user provides col_types as a vector, col_names can have one entry per column, i.e. have the same length as col_types, or one entry per unskipped column."
col_types,"Either NULL to guess all from the spreadsheet or a character vector containing one entry per column from these options: ""skip"", ""guess"", ""logical"", ""numeric"", ""date"", ""text"" or ""list"". If exactly one col_type is specified, it will be recycled. The content of a cell in a skipped column is never read and that column will not appear in the data frame output. A list cell loads a column as a list of length 1 vectors, which are typed using the type guessing logic from col_types = NULL, but on a cell-by-cell basis."
na,"Character vector of strings to interpret as missing values. By default, readxl treats blank cells as missing data."
trim_ws,Should leading and trailing whitespace be trimmed?
skip,"Minimum number of rows to skip before reading anything, be it column names or data. Leading empty rows are automatically skipped, so this is a lower bound. Ignored if range is given."
n_max,"Maximum number of data rows to read. Trailing empty rows are automatically skipped, so this is an upper bound on the number of rows in the returned tibble. Ignored if range is given."
guess_max,Maximum number of data rows to use for guessing column types.
